In [1]:
# Importa find_best_hyperparameter e pandas
from find_best_hyperparameters import AtividadeTres

# Carrega Atividade 3
a3 = AtividadeTres(random_state=77)

# Define hiper parâmetros para Greedy Search
hyper_params = {}

hyper_params['nb'] = {
    'model_name': 'MultinomialNB',
    'params': a3.create_param_grid({   
        # Explicar alpha
        'alpha': [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0],
        # Aprender probabilidades anteriores de classe
        'fit_prior': [True, False]
    })
}

hyper_params['lr'] = {
    'model_name': 'LogisticRegression',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.1, 0.25, 1.0, 5.0, 10.0, 20.0],
        # Algoritmos de otimização
        'solver': ['lbfgs', 'liblinear', 'sag'],
        # Tipos de regularização
        'penalty': ['l1', 'l2', 'elasticnet', None],
        # Iterações máximas
        'max_iter': [100, 200, 500, 1000]
    })
}

hyper_params['svm'] = {
    'model_name': 'LinearSVC',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.01, 0.1, 1, 5, 10, 50, 100],
        # Função de perda
        'loss': ['hinge', 'squared_hinge'],
        # Iterações máximas
        'max_iter': [100, 250, 500, 1000, 5000],
        # Critério de tolerância para parar
        'tol': [1e-3, 1e-4, 1e-5]
    })
}

Classe iniciada com semente 77!
Hiper parâmetros montados: [{'alpha': 0.1, 'fit_prior': True}, {'alpha': 0.1, 'fit_prior': False}, {'alpha': 0.25, 'fit_prior': True}, {'alpha': 0.25, 'fit_prior': False}, {'alpha': 0.5, 'fit_prior': True}, {'alpha': 0.5, 'fit_prior': False}, {'alpha': 0.75, 'fit_prior': True}, {'alpha': 0.75, 'fit_prior': False}, {'alpha': 1.0, 'fit_prior': True}, {'alpha': 1.0, 'fit_prior': False}, {'alpha': 2.0, 'fit_prior': True}, {'alpha': 2.0, 'fit_prior': False}, {'alpha': 5.0, 'fit_prior': True}, {'alpha': 5.0, 'fit_prior': False}]
Hiper parâmetros montados: [{'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l1', 'max_iter': 100}, {'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l1', 'max_iter': 200}, {'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l1', 'max_iter': 500}, {'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l1', 'max_iter': 1000}, {'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 100}, {'C': 0.1, 'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 200}, {'C': 0.1, 'solver':

In [2]:
# Leitura e processamento da base CSTR
BASE='Dmoz'
FILENAME=BASE + '-Computers.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

,file_name,text,class
0,Intelligence_1434272.txt,Principal Curves Page Introduction to principa...,Artificial
1,Intelligence_1433615.txt,Iowa State University - Artificial Intelligenc...,Artificial
2,Intelligence_1434400.txt,"Paccanaro, Alberto Learning distributed repres...",Artificial
3,Intelligence_1434281.txt,Attrasoft Provider of a number of neural netwo...,Artificial
4,Intelligence_1434141.txt,LibML A machine learning library. New implemen...,Artificial


In [3]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

Dados divididos com tamanho do teste 0.20 e semente 77!
Dicionário de séries: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/Dmoz_train.csv salvo com sucesso!
Arquivo out/Dmoz_test.csv salvo com sucesso!


,text,class
2867,"Franklin Telecom Corp. Designs, manufactures, ...",Data
7309,SteeleSoft Consulting A web development compan...,Programming
8544,Plot Maestro A plotting program but also allow...,Software
8382,Secward Technologies Inc. Encryption products ...,Security
7559,AiboPet's Reborn Site AIBO customization site ...,Robotics


In [4]:
# Greedy Search para NB, LR and SVM
import pandas as pd

best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    best_greedy[item] = greedy.max().to_dict()

# Exibe valores no notebook
pd.DataFrame(best_greedy)

Arquivo out/Dmoz_nb.csv salvo com sucesso!
Arquivo out/Dmoz_lr.csv salvo com sucesso!
Arquivo out/Dmoz_svm.csv salvo com sucesso!


,nb,lr,svm
alpha,5.0,NaN,NaN
fit_prior,True,NaN,NaN
score,0.964211,0.996711,0.994737
C,NaN,1.0,0.1
solver,NaN,sag,NaN
penalty,NaN,l2,NaN
max_iter,NaN,1000,5000
loss,NaN,NaN,squared_hinge
tol,NaN,NaN,0.001
